In [1]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier,VotingRegressor
import pandas as pd
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, accuracy_score, roc_curve, roc_auc_score,log_loss,classification_report
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.compose import make_column_transformer, make_column_selector
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeRegressor




In [4]:
in_df = pd.read_csv('insurance.csv')

In [6]:
in_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [8]:
X = in_df.drop('charges',axis=1 )
y = in_df['charges']

In [10]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=24,test_size=0.3)

In [50]:
ohe = OneHotEncoder(handle_unknown = 'ignore', sparse_output = False).set_output(transform = 'pandas')
ct_ohe = make_column_transformer(('passthrough',make_column_selector(dtype_exclude = object) ),
                                  (ohe,make_column_selector(dtype_include = object) ),
                                  verbose_feature_names_out = False).set_output(transform = 'pandas')

dtc = DecisionTreeRegressor()
lr = LinearRegression()
el = ElasticNet()

In [67]:
pipe_lr = Pipeline([('CT',ct_ohe),('LR',lr)])
pipe_lr.fit(X_train,y_train)
y_pred = pipe_lr.predict(X_test)
lr_wt = r2_score(y_test,y_pred)
lr_wt

0.7665391799816873

In [69]:
pipe_el = Pipeline([('CT',ct_ohe),('EL',el)])
pipe_el.fit(X_train,y_train)
y_pred = pipe_el.predict(X_test)
el_wt = r2_score(y_test,y_pred)
el_wt

0.49673651151300835

In [71]:
pipe_dtc = Pipeline([('CT',ct_ohe),('DTC',dtc)])
pipe_dtc.fit(X_train,y_train)
y_pred = pipe_dtc.predict(X_test)
dt_wt = r2_score(y_test,y_pred)
dt_wt

0.7066703808007823

In [73]:
voting = VotingRegressor([('LR',pipe_lr),('EL',pipe_el),('DT',pipe_dtc)])
voting.fit(X_train,y_train)
y_pred = voting.predict(X_test)
r2_score(y_test,y_pred)


0.7745012134228847

## weighted

In [78]:
voting = VotingRegressor([('LR',pipe_lr),('EL',pipe_el),('DT',pipe_dtc)],
                        weights = [lr_wt,el_wt,dt_wt])
voting.fit(X_train,y_train)
y_pred = voting.predict(X_test)
r2_score(y_test, y_pred)

0.7877625747926207

## GridSearchCV